# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import os
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,kharkhorin,47.1975,102.8238,14.58,75,80,4.39,MN,1721593471
1,west island,-12.1568,96.8225,26.99,89,75,5.66,CC,1721593473
2,blackmans bay,-43.0167,147.3167,10.29,87,100,1.38,AU,1721593475
3,adamstown,-25.0660,-130.1015,18.83,66,95,8.71,PN,1721593478
4,waitangi,-43.9535,-176.5597,9.90,99,100,4.02,NZ,1721593480


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 800,
    frame_height = 400,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha =0.5,
    tiles='OSM')
# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)
    & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
53,jamestown,42.0970,-79.2353,26.77,46,0,0.00,US,1721593582
73,kyzylorda,44.8528,65.5092,22.01,49,0,2.00,KZ,1721593625
96,lac-brome,45.2168,-72.5158,23.54,51,0,2.92,CA,1721593670
123,badger,64.8000,-147.5333,23.88,52,0,0.00,US,1721593720
169,port-saint-louis-du-rhone,43.3833,4.8000,22.13,73,0,4.12,FR,1721593802
170,colonia,50.9333,6.9500,21.92,78,0,3.09,DE,1721593804
189,lata,40.1629,-8.3327,22.73,85,0,0.98,PT,1721593841
206,melfort,52.8667,-104.6177,26.07,65,0,1.54,CA,1721593875
221,vadso,70.0744,29.7487,23.49,68,0,2.06,NO,1721593903


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
53,jamestown,US,42.0970,-79.2353,46,
73,kyzylorda,KZ,44.8528,65.5092,49,
96,lac-brome,CA,45.2168,-72.5158,51,
123,badger,US,64.8000,-147.5333,52,
169,port-saint-louis-du-rhone,FR,43.3833,4.8000,73,
170,colonia,DE,50.9333,6.9500,78,
189,lata,PT,40.1629,-8.3327,85,
206,melfort,CA,52.8667,-104.6177,65,
221,vadso,NO,70.0744,29.7487,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel", "apiKey": geoapify_key, "limit":20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: DoubleTree Jamestown
kyzylorda - nearest hotel: Нур
lac-brome - nearest hotel: Auberge Knowlton
badger - nearest hotel: Clarion Hotel & Suites
port-saint-louis-du-rhone - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
lata - nearest hotel: Residencial Botânico
melfort - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
da nang - nearest hotel: Thanh Thanh Hotel
balykshi - nearest hotel: Арасан
capitan bado - nearest hotel: No hotel found
myrina - nearest hotel: Ήφαιστος
naxos - nearest hotel: Chateau Zevgoli
tres lagoas - nearest hotel: Hotel Sul
jaragua - nearest hotel: Boa Vista Park Hotel
zhezqazghan - nearest hotel: No hotel found
balaklava - nearest hotel: Рыбацкая слобода
archangelos - nearest hotel: Semina


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
53,jamestown,US,42.0970,-79.2353,46,DoubleTree Jamestown
73,kyzylorda,KZ,44.8528,65.5092,49,Нур
96,lac-brome,CA,45.2168,-72.5158,51,Auberge Knowlton
123,badger,US,64.8000,-147.5333,52,Clarion Hotel & Suites
169,port-saint-louis-du-rhone,FR,43.3833,4.8000,73,No hotel found
170,colonia,DE,50.9333,6.9500,78,Wasserturm Hotel Cologne
189,lata,PT,40.1629,-8.3327,85,Residencial Botânico
206,melfort,CA,52.8667,-104.6177,65,No hotel found
221,vadso,NO,70.0744,29.7487,68,Scandic Vadsø


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 800,
    frame_height = 400,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha =0.5,
    tiles='OSM',
    hover_cols =["Hotel Name","Country"])

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)